<a href="https://colab.research.google.com/github/ivnlee/langchain-course/blob/main/langchain_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain & Vector Databases**

### **Module 1: From Zero to Hero**

In [ ]:
!pip install langchain==0.0.208 deeplake openai tiktoken
!pip install python-dotenv

In [11]:
from dotenv import load_dotenv

load_dotenv('api_key.env')

True

### **The LLMs**



#### The fundamental component of LangChain involves invoking an LLM with a specific input.

#### To accomplish this, we will first need to import the LLM wrapper.

In [3]:
from langchain.llms import OpenAI

In [4]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
llm = OpenAI(model="text-davinci-003", temperature=0.9)

In [5]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



1. Jogging: Start with a 15-minute jog, increasing gradually to a 30-minute jog 3-4 times a week.

2. Cycling: Cycle for 30 minutes 3-4 times a week, gradually increasing the duration as you get more fit.

3. Swimming: Swim laps for 30 minutes, 3-4 times a week. Start with a light level of intensity and then increase as you get more fit.

4. Hill running: Run up & down hills for 15 minutes 2-3 times a week, increasing the intensity and duration as you get more fit.

5. Interval training: Aside from your regular cardio routine, do 2-3 days of interval training. Start with a 10-minute warm up, then do 1 minute of sprinting followed by 1 minute of walking. Repeat for a total of 20 minutes.

6. Strength training: Do strength training exercises at least once a week to improve your overall strength, which will help you with your cardiovascular endurance. 

7. Outdoor activities: Incorporate more outdoor activities into your routine. Go rock climbing, take a hike, or join a local sports tea

### **The Chains**

#### In LangChain, a chain is an end-to-end wrapper around multiple individual components, providing a way to accomplish a common use case by combining these components in a specific sequence.

The most commonly used type of chain is the `LLMChain`, which consists of a `PromptTemplate`, a model (either an LLM or a ChatModel), and an optional output parser.

In the next example, we demonstrate how to create a chain that generates a possible name for a company that produces eco-friendly water bottles. By using LangChain's `LLMChain`, `PromptTemplate`, and `OpenAI` classes, we can easily define our prompt, set the input variables, and generate creative outputs.

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = OpenAI(model="text-davinci-003", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))



EcoPure Bottles


### **The Memory**

#### In LangChain, Memory refers to the mechanism that stores and manages the conversation history between a user and the AI. It helps maintain context and coherency throughout the interaction, enabling the AI to generate more relevant and accurate responses.

In [8]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [9]:
llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!
Human: What can yo



---



### **Deep Lake VectorStore**

In [ ]:
!pip install deeplake

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [13]:
# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
my_activeloop_org_id = "ivanlee"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"

dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


/

Dataset(path='hub://ivanlee/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['fa0d48b6-1a21-11ee-bb9d-0242ac1c000c',
 'fa0d4ad2-1a21-11ee-bb9d-0242ac1c000c']

Now, let's create a `RetrievalQA` chain:

In [14]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

Next, let's create an agent that uses the `RetrievalQA` chain as a tool:

In [15]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

Finally, we can use the agent to ask a question:

In [16]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.
